In [17]:
from qiskit import IBMQ
import pennylane_qiskit 
import pennylane as qml
from pennylane import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService

#IBMQ.save_account("", overwrite=True)
#IBMQ.load_account() # Load account from disk
#provider = IBMQ.get_provider(hub='qhack-event', group='main', project='level-1')
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# Initialize the account first.
service = QiskitRuntimeService(channel='ibm_quantum', 
                               token='')
services = service.backends()
print(services)

[<IBMBackend('ibmq_jakarta')>, <IBMBackend('ibm_nairobi')>, <IBMBackend('ibmq_qasm_simulator')>, <IBMBackend('ibmq_lima')>, <IBMBackend('ibmq_belem')>, <IBMBackend('ibmq_quito')>, <IBMBackend('simulator_statevector')>, <IBMBackend('simulator_mps')>, <IBMBackend('simulator_extended_stabilizer')>, <IBMBackend('simulator_stabilizer')>, <IBMBackend('ibmq_manila')>, <IBMBackend('ibm_lagos')>, <IBMBackend('ibm_perth')>, <IBMBackend('ibm_oslo')>]


In [4]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Optimization of molecular geometries
====================================

::: {.meta}
:property=\"og:description\": Find the equilibrium geometry of a
molecule :property=\"og:image\":
<https://pennylane.ai/qml/_images/fig_pes.png>
:::

::: {.related}
tutorial\_quantum\_chemistry Building molecular Hamiltonians
tutorial\_vqe A brief overview of VQE tutorial\_givens\_rotations Givens
rotations for quantum chemistry
:::

*Author: Alain Delgado --- Posted: 30 June 2021. Last updated: 25 June
2022.*

Predicting the most stable arrangement of atoms in a molecule is one of
the most important tasks in quantum chemistry. Essentially, this is an
optimization problem where the total energy of the molecule is minimized
with respect to the positions of the atomic nuclei. The molecular
geometry obtained from this calculation is in fact the starting point
for many simulations of molecular properties. If the geometry is
inaccurate, then any calculations that rely on it may also be
inaccurate.

Since the nuclei are much heavier than the electrons, we can treat them
as point particles clamped to their positions. Under this assumption,
the total energy of the molecule $E(x)$ depends on the nuclear
coordinates $x$, which define the potential energy surface. Solving the
stationary problem $\nabla_x E(x) = 0$ corresponds to molecular geometry
optimization and the optimized nuclear coordinates determine the
equilibrium geometry of the molecule. The figure below illustrates these
concepts for the [trihydrogen
cation](https://en.wikipedia.org/wiki/Trihydrogen_cation). Its
equilibrium geometry in the electronic ground state corresponds to the
minimum energy of the potential energy surface. At this minimum, the
three hydrogen atoms are located at the vertices of an equilateral
triangle whose side length is the optimized bond length $d$.

| 

![](/demonstrations/mol_geo_opt/fig_pes.png){.align-center
width="50.0%"}

| 

In this tutorial, you will learn how to recast the problem of finding
the equilibrium geometry of a molecule in terms of a general variational
quantum algorithm. The central idea is to consider explicitly that the
target electronic Hamiltonian $H(x)$ is a **parametrized** observable
that depends on the nuclear coordinates $x$. This implies that the
objective function, defined by the expectation value of the Hamiltonian
computed in the trial state prepared by a quantum computer, depends on
both the quantum circuit and the Hamiltonian parameters.

The quantum algorithm in a nutshell
-----------------------------------

The goal of the variational algorithm is to find the global minimum of
the cost function
$g(\theta, x) = \langle \Psi(\theta) \vert H(x) \vert \Psi(\theta) \rangle$
with respect to the circuit parameters $\theta$ and the nuclear
coordinates $x$ entering the electronic Hamiltonian of the molecule. To
that end, we use a gradient-descent method and follow a **joint**
optimization scheme where the gradients of the cost function with
respect to circuit and Hamiltonian parameters are simultaneously
computed at each step. This approach does not require nested
optimization of the state parameters for each set of nuclear
coordinates, as occurs in classical algorithms for molecular geometry
optimization, where the energy minimum is searched for along the
potential energy surface of the electronic state.

In this tutorial we demonstrate how to use PennyLane to implement
quantum optimization of molecular geometries. The algorithm consists of
the following steps:

1.  Build the parametrized electronic Hamiltonian $H(x)$ of the
    molecule.

2.  Design the variational quantum circuit to prepare the electronic
    trial state of the molecule, $\vert \Psi(\theta) \rangle$.

3.  Define the cost function
    $g(\theta, x) = \langle \Psi(\theta) \vert H(x) \vert
    \Psi(\theta) \rangle$.

4.  Initialize the variational parameters $\theta$ and $x$. Perform a
    joint optimization of the circuit and Hamiltonian parameters to
    minimize the cost function $g(\theta, x)$. The gradient with respect
    to the circuit parameters can be obtained using a variety of
    established methods, which are natively supported in PennyLane. The
    gradients with respect to the nuclear coordinates can be computed
    using the formula

    $$\nabla_x g(\theta, x) = \langle \Psi(\theta) \vert \nabla_x H(x) \vert \Psi(\theta) \rangle.$$

Once the optimization is finalized, the circuit parameters determine the
energy of the electronic state, and the nuclear coordinates determine
the equilibrium geometry of the molecule in this state.

Let\'s get started! ⚛️

Building the parametrized electronic Hamiltonian
------------------------------------------------

In this example, we want to optimize the geometry of the berilium dihydride $\mathrm{BeH}_2$, described in a minimal basis set, where two
electrons are shared between three hydrogen atoms (see figure above).
The molecule is specified by providing a list with the symbols of the
atomic species and a one-dimensional array with the initial set of
nuclear coordinates in [atomic
units](https://en.wikipedia.org/wiki/Hartree_atomic_units) .


In [6]:
import pennylane as qml
from pennylane import numpy as np

symbols = ["H", "Be", "H"]
geometry = np.array([[0.0, 0.0, -1.3441],
                     [0.0, 0.0, 0.0],
                     [0.0, 0.0, 1.3441]], requires_grad=True)

Next, we need to build the parametrized electronic Hamiltonian $H(x)$.
We use the Jordan-Wigner transformation to represent the fermionic
Hamiltonian as a linear combination of Pauli operators,

$$H(x) = \sum_j h_j(x) \prod_i^{N} \sigma_i^{(j)}.$$

The expansion coefficients $h_j(x)$ carry the dependence on the
coordinates $x$, the operators $\sigma_i$ represent the Pauli group
$\{I, X, Y, Z\}$, and $N$ is the number of qubits required to represent
the electronic wave function.

We define the function `H(x)` to build the parametrized Hamiltonian of
the trihydrogen cation using the
`~.pennylane.qchem.molecular_hamiltonian`{.interpreted-text role="func"}
function.


In [7]:
H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, basis='sto-3g', active_electrons=2, active_orbitals=4)

In [8]:
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators, qubits)

for idx, generator in enumerate(generators):
    print(f"generator {idx+1}: {generator}, paulix_op: {paulixops[idx]}")

generator 1:   (1.0) [Z2 Z3], paulix_op: PauliX(wires=[3])
generator 2:   (1.0) [Z4 Z5], paulix_op: PauliX(wires=[5])
generator 3:   (1.0) [Z0 Z2 Z4 Z6], paulix_op: PauliX(wires=[6])
generator 4:   (1.0) [Z1 Z2 Z4 Z7], paulix_op: PauliX(wires=[7])


/opt/conda/lib/python3.8/site-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


In [9]:
n_electrons = 2
paulix_sector = qml.qchem.optimal_sector(H, generators, n_electrons)
print(paulix_sector)

[1, 1, -1, -1]


In [ ]:
H_tapered = qml.taper(H, generators, paulixops, paulix_sector)
print(H_tapered)

  ((-12.728024776058982+0j)) [I0]
+ ((-0.7558586515477563+0j)) [Z4]
+ ((-0.7558586515474329+0j)) [Z2]
+ ((-0.21380711646194148+0j)) [Z1]
+ ((-0.2138071164619414+0j)) [Z0]
+ ((-0.001993018450980797+0j)) [X0]
+ ((0.001993018450980797+0j)) [X1]
+ ((0.009413379410957741+0j)) [X2]
+ ((0.009413379410962921+0j)) [X4]
+ ((-0.0440553980076435+0j)) [X0 Z1]
+ ((-0.035479687877840574+0j)) [Y0 Y1]
+ ((-0.010289153698466596+0j)) [Y1 Y4]
+ ((-0.010289153698466596+0j)) [X1 X4]
+ ((-0.010289153698460975+0j)) [Y1 Y2]
+ ((-0.010289153698460975+0j)) [X1 X2]
+ ((0.009509419594518606+0j)) [Z0 Z2]
+ ((0.009509419594518606+0j)) [Z1 Z2]
+ ((0.009509419594727855+0j)) [Z0 Z4]
+ ((0.009509419594727855+0j)) [Z1 Z4]
+ ((0.010289153698460975-0j)) [X0 X2]
+ ((0.010289153698460975-0j)) [Y0 Y2]
+ ((0.010289153698466596-0j)) [X0 X4]
+ ((0.010289153698466596-0j)) [Y0 Y4]
+ ((0.01212469133649474+0j)) [Y2 Y4]
+ ((0.01212469133649474+0j)) [X2 X4]
+ ((0.0440553980076435-0j)) [Z0 X1]
+ ((0.20095706463501226+0j)) [Z2 Z4]
+ ((0

In [ ]:
print(H.wires)
print(H_tapered.wires)

<Wires = [0, 1, 2, 3, 4, 5, 6, 7]>
<Wires = [0, 1, 2, 4]>


In [ ]:
H_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H), wires=H.wires)
H_tapered_sparse = qml.SparseHamiltonian(qml.utils.sparse_hamiltonian(H_tapered), wires=H_tapered.wires)

print("Eigenvalues of H:\n", qml.eigvals(H_sparse, k=8))
print("\nEigenvalues of H_tapered:\n", qml.eigvals(H_tapered_sparse, k=2))

Eigenvalues of H:
 [-14.89645067 -14.73783044 -14.73783044 -14.6172819  -14.6172819
 -14.73783044 -14.73783044 -14.6172819 ]

Eigenvalues of H_tapered:
 [-14.89645067 -14.16700922]


In [ ]:
state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons, num_wires=len(H.wires))
print(state_tapered,type(state_tapered))

[1 1 0 0] <class 'pennylane.numpy.tensor.tensor'>


Recall that the original Hartree-Fock state for the BeH2 is [1111000] for 4 electrons and [1100] for 2 electrons. We can now generate the qubit representation of these states and compute the Hartree-Fock energies for each Hamiltonian.

In [ ]:
dev = qml.device("default.qubit", wires=H.wires)
#dev = qml.device('qiskit.ibmq', wires=H.wires, backend='ibm_nairobi', provider=provider)
@qml.qnode(dev)
def circuit():
    qml.BasisState(np.array([1, 1, 0, 0, 0, 0, 0, 0]), wires=H.wires)
    return qml.state()

qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H).toarray() @ qubit_state
print(f"HF energy: {np.real(HF_energy):.8f} Ha")

dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def circuit():
    qml.BasisState(np.array([ 1, 1, 0, 0]), wires=H_tapered.wires)
    return qml.state()

qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H_tapered).toarray() @ qubit_state
print(f"HF energy (tapered): {np.real(HF_energy):.8f} Ha")

HF energy: -14.88288874 Ha
HF energy (tapered): -14.88288874 Ha


Energy for 6-31g basis set is:

HF energy: -15.12050563 Ha

HF energy (tapered): -15.12050563 Ha

These values are close to the reference Hartree-Fock energy −15.773 Ha. [DOI:10.1021/j100868a076, DOI:10.1103/PhysRevResearch.3.013104, DOI:10.1002/(SICI)1097-461X(1996)60:1%3C493::AID-QUA48%3E3.0.CO;2-A]

### VQE Simulation

In [ ]:
singles, doubles = qml.qchem.excitations(n_electrons, len(H.wires))
tapered_doubles = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=double) for double in doubles
]
tapered_singles = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=single) for single in singles
]


In [36]:
dev = qml.device("default.qubit", wires=H_tapered.wires)
#dev = pennylane_qiskit.ibmq.IBMQDevice(wires=H_tapered.wires, provider=None, backend='ibmq_qasm_simulator', shots=None)
#dev = qml.device('qiskit.ibmq', wires=H_tapered.wires, backend='ibmq_lima', provider=provider)
#dev = qml.device('qiskit.aer', wires=H_tapered.wires, backend="statevector_simulator", shots=300)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)


In [37]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
#optimizer = qml.QNGOptimizer(stepsize=0.5)
#optimizer = qml.ShotAdaptiveOptimizer(min_shots=10)
#optimizer = opt = qml.AdamOptimizer(0.05)
params = np.zeros(len(doubles) + len(singles), requires_grad=True)

for n in range(1, 45):
    params, energy = optimizer.step_and_cost(tapered_circuit, params)
    if not n % 5:
        print(f"n: {n}, E: {energy:.8f} Ha")

n: 5, E: -14.89456179 Ha
n: 10, E: -14.89622374 Ha
n: 15, E: -14.89642194 Ha
n: 20, E: -14.89644694 Ha
n: 25, E: -14.89645018 Ha
n: 30, E: -14.89645060 Ha
n: 35, E: -14.89645066 Ha
n: 40, E: -14.89645067 Ha


## Single point energy computation

In [24]:
dev = qml.device('qiskit.ibmq', wires=H_tapered.wires, backend='ibmq_lima', provider=provider, shots=1000)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)

In [25]:
# Energy from IMB QPU
energy = tapered_circuit(params)

print(energy)

(-13.4063866589651+0j)


The computed energy is close to the experimental ground state energy, −15.4 Ha, while the number of qubits and the number of Hamiltonian terms are significantly reduced with respect to their original values.

In [21]:
from pennylane_qiskit import upload_vqe_runner, vqe_runner

#BMQ.enable_account(token="")
#BMQ.load_account() # Load account from disk
#IBMQ.get_provider(hub='ibm-q')

rogram_id = upload_vqe_runner(hub="qhack-event", group="main", project="level-1")

def vqe_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)


params = np.zeros(len(doubles) + len(singles), requires_grad=True)
hamiltonian = H_tapered

job = vqe_runner(
    program_id=program_id,
    backend="ibmq_qasm_simulator",
    hamiltonian=hamiltonian,
    ansatz=vqe_circuit,
    x0=params,
    shots=100,
    optimizer="SPSA",
    optimizer_config={"maxiter": 55},
    #kwargs={"hub": "qhack-event", "group": "main", "project": "level-1"},
)


job.result()
results = job.intermediate_results['function']

Traceback (most recent call last):
  Input In [21] in <cell line: 7>
    program_id = upload_vqe_runner(hub="ibm-q", group="open", project="main")
  File /opt/conda/lib/python3.8/site-packages/pennylane_qiskit/vqe_runtime_runner.py:119 in upload_vqe_runner
    connect(kwargs)
  File /opt/conda/lib/python3.8/site-packages/pennylane_qiskit/ibmq.py:136 in connect
    IBMQ.load_account()
  File /opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:191 in load_account
    self._initialize_providers(credentials, preferences)
  File /opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:456 in _initialize_providers
    auth_client = AuthClient(credentials.token,
  File /opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/api/clients/auth.py:41 in __init__
    self.base_api = self._init_service_clients(**request_kwargs)
  File /opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/api/clients/auth.py:53 in _init_service_clients
    acces